# Free form text to geographical information

What is this notebook? I needed a way to figure out in which country are users of Stack Overflow based. Stack overflow collect this in free text form, so its highly non-standardised and people can write anything. This notebook attempts to deal with this problem.

Some solutions are hacky and I needed things done quick and dirty. Don't @ me :D

In [2]:
import pandas as pd
import numpy as np
import geograpy as gg
import spacy
import en_core_web_sm
from geopy.geocoders import Nominatim

## Import raw(ish) stack overflow user data 

In [3]:
activeusers = pd.read_csv('data_csv/active_users.csv', index_col = 0)

In [4]:
activeusers.shape

(984275, 22)

In [109]:
activeusers.head()

,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,...,HasWebsite,HasBio,HasLocation,HasProfileImage,AboutMeClean,AboutMeLinks,Seniority,Student,AboutMeLength,UsaBased
Id,,,,,,,,,,,,,,,,,,,,,
1,59111,2008-07-31T14:22:31Z,Jeff Atwood,2020-05-02T18:23:48Z,http://www.codinghorror.com/blog/,"El Cerrito, CA","<p><a href=""http://www.codinghorror.com/blog/a...",548898,3378,1311,...,1,1,1,1,Stack Overflow Valued Associate #00001 Wonder...,5.0,False,False,54.0,True
2,5632,2008-07-31T14:22:31Z,Geoff Dalgas,2020-05-30T06:34:16Z,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the Stack Overflow team. Find...,26613,664,88,...,1,1,1,1,Developer on the Stack Overflow team. Find me...,2.0,False,False,15.0,True
3,15196,2008-07-31T14:22:31Z,Jarrod Dixon,2020-05-29T15:37:16Z,http://jarroddixon.com,"Raleigh, NC, United States","<p><a href=""http://blog.stackoverflow.com/2009...",26423,7756,100,...,1,1,1,1,Developer on the Stack Overflow team. Was dub...,2.0,False,False,38.0,True
4,31887,2008-07-31T14:22:31Z,Joel Spolsky,2020-05-30T17:25:45Z,https://joelonsoftware.com/,"New York, NY","<p>In 2000 I co-founded Fog Creek Software, wh...",78047,825,97,...,1,1,1,1,"In 2000 I co-founded Fog Creek Software, where...",6.0,True,False,121.0,True
5,48438,2008-07-31T14:22:31Z,Jon Galloway,2020-05-29T23:45:55Z,http://weblogs.asp.net/jgalloway/,"San Diego, CA","<p>Technical Evangelist at Microsoft, speciali...",13046,786,34,...,1,1,1,0,"Technical Evangelist at Microsoft, specializin...",0.0,False,False,29.0,True


In [111]:
activeusers[activeusers.Location.isna()].shape

(492877, 22)

# Geopy

Queries Open Street Maps and returns best fit for longitude, latitude, country and country code

In [6]:
# Choose nominatim as your OSM query mechanism
geolocator = Nominatim(user_agent="andreas_research")

In [142]:
# This is an example of how it works
place_str = 'Jurija Gagarina Novi Beograd'
location = geolocator.geocode(place_str)

location = geolocator.reverse([location.latitude, location.longitude])

location.raw

{'place_id': 197319429,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 517989957,
 'lat': '44.7977944',
 'lon': '20.371818225000002',
 'display_name': '231b, Јурија Гагарина, Сава, Блок 45, Градска општина Нови Београд, Београд, Град Београд, Централна Србија, 11077, Србија (Serbia)',
 'address': {'house_number': '231b',
  'road': 'Јурија Гагарина',
  'administrative': 'Сава',
  'residential': 'Блок 45',
  'suburb': 'Градска општина Нови Београд',
  'city': 'Београд',
  'county': 'Град Београд',
  'state': 'Централна Србија',
  'postcode': '11077',
  'country': 'Србија (Serbia)',
  'country_code': 'rs'},
 'boundingbox': ['44.7977382', '44.7978134', '20.3717505', '20.3718632']}

### Warning: the code below executes at the rate of 1 second per row. That can take a long time

In [ ]:
location_lookup = []

# go through all unique locations in the dataset
for Location in activeusers.Location.unique():
    try:
        
        # Queries OSM
        loc = geolocator.geocode(Location)
        
        if loc is not None:
            #gets the location information
            loc = geolocator.reverse([loc.latitude, loc.longitude])
            
            #adds relevant fields to the list
            location_lookup.append([Location, loc.latitude, 
                                    loc.longitude, loc.raw['address']['country'], 
                                    loc.raw['address']['country_code']])
        
        else:
            # if it can't find the location
            location_lookup.append([Location, '', '', '', ''])
    
    except:
        # catches occasional exceptions
        location_lookup.append([Location, 'exception', '', '', ''])
        
# Save as dataframe
loc_df = pd.DataFrame(location_lookup, columns=['Location', 'latitude', 
                                                'longitude', 'country', 'code'])

In [ ]:
# force correcting NaN being considered a town in Italy
loc_df.iloc[5].latitude = ''
loc_df.iloc[5].longitude = ''
loc_df.iloc[5].country = ''
loc_df.iloc[5].code = ''

In [145]:
loc_df[loc_df.code == '']

,original,latitude,longitude,country,code
5,NaN,,,,
128,From up there. Now down here.,,,,
188,"Greater Perthshire, Scotland",,,,
413,Omnipresent,,,,
481,"San Diego Metropolitan Area, San Diego County,...",,,,
...,...,...,...,...,...
44869,"Cianjur, Cianjur Regency, West Java, Indonesia",,,,
44877,"Yorkshire Norte, UK",,,,
44878,"The Lost City of Atlantis, Somewhere in the World",,,,
44885,Moving Part U-Designate-0,,,,


## Trying to locate places not successfully done previously

Some locations were not successfully found by OSM - either because they aren't locations ("From up there. Now down here") or because they aren't spelled correctly, contain multiple places...

The code below attempts to go through those and figure out a better term to search for. This will lower our precision in general but it should be correct in most cases, as I am predominantly interested in a country someone is from.

In [ ]:
# list to keep the result
tricky_thingies = []

nlp = en_core_web_sm.load()

# list of world cities with countires and subcountries
world_cities = pd.read_csv('data_csv/world_cities.csv', index_col = 0)

# Iterate through all locations which OSM couldn't figure out
for index, row in loc_df[loc_df.code == ''].iterrows():
    
    # placeholder to keep the best result
    best_indicator = ''
    
    if type(row.Location) == str:
            
        doc = nlp(row.Location)
        
        # for each entity in the location text, check if its a country, subcountry or city
        for ent in doc.ents:
            
            if ent.text in world_cities.country.unique():
                best_indicator = ent.text
                break
                
            elif ent.text in world_cities.subcountry.unique():
                best_indicator = ent.text
                break
                
            elif ent.text in world_cities.index:
                # the reason why we are not breaking out of the loop here is because
                # multiple countries could have a city called e.g. "Belgrade", so I prefer
                # to look further if there is a better indicator than city
                best_indicator = ent.text
        
        # if previous code still didn't fix our issue, we go in
        # example that would be caught below but not above:
        # "Thessaloniki, Kentriki Makedonia Greece"
        if best_indicator == '':
            
            # gets us a clean list containing all phrases, and then all words
            words_dirty = row.Location.split(',') + row.Location.split(' ')
            words_clean = [re.sub(r'[^\w\s]','', word.strip()) for word in words_dirty]
            
            for word in words_clean:
                
                if word in world_cities.country.unique():
                    best_indicator = word
                    break
                    
                elif word in world_cities.subcountry.unique():
                    best_indicator = word
                    break
                    
                elif word in world_cities.index:
                    best_indicator = word
                
                
    tricky_thingies.append([row.Location, best_indicator])
    
tricky_df = pd.DataFrame(tricky_thingies, columns=['Location', 'search_term'])

In [152]:
tricky_df.head(10)

,original,search_term
0,NaN,
1,From up there. Now down here.,
2,"Greater Perthshire, Scotland",Scotland
3,Omnipresent,
4,"San Diego Metropolitan Area, San Diego County,...",United States
5,"Roverdo, Switzerland",Switzerland
6,Duchy of Grand Fenwick,
7,127.0.0.1,
8,"Columbus, OH/Atlanta, GA",Columbus
9,"Thessaloniki, Kentriki Makedonia Greece",Greece


In [ ]:
# now that we have better search terms, we go and query OSM again
tricky_locs = []

for index, row in tricky_df[tricky_df.search_term != ''].iterrows():
    try:
        loc = geolocator.geocode(row.search_term)
        
        if loc is not None:
            loc = geolocator.reverse([loc.latitude, loc.longitude])
            tricky_locs.append([row.Location, loc.latitude, 
                                loc.longitude, loc.raw['address']['country'], 
                                loc.raw['address']['country_code']])
        else:
            tricky_locs.append([row.Location, '', '', '', ''])
    except:
        tricky_locs.append([row.Location, 'exception', '', '', ''])
        
tricky_solved_df = pd.DataFrame(tricky_locs, columns=['Location', 'latitude', 'longitude', 'country', 'code'])

In [155]:
tricky_solved_df[tricky_solved_df.code == '']

,Location,latitude,longitude,country,code
144,Cayman Islands,exception,,,
822,"Qaraghandy, Kazakhstan",,,,
1517,"El Jadida, Doukkala-Abda, Morocco",,,,
1569,Qaraghandy,,,,
1587,"Pandalan Island, Pandami, Autonomous Region in...",,,,
1740,"Staryy Oskol, Белгородская область, Россия",,,,
2053,"Qaraghandy, Karagandy Province, Kazakhstan",,,,


In [124]:
# Merge the two location dataframes we now have
merged_locations = pd.concat([loc_df[loc_df.code != ''], tricky_solved_df], ignore_index=True)

In [157]:
merged_locations.sample(20)

,Location,latitude,longitude,country,code
16887,"Senigallia, AN, Italia",43.7148,13.2177,Italia,it
24573,"Calabasas Hills, Calabasas, CA, United States",34.132,-118.711,United States of America,us
15665,"Gabrovo, Bulgaria",42.9616,25.2401,България,bg
11582,Santa Monica,34.0252,-118.496,United States of America,us
14662,"San Francisco, California, EE. UU.",37.779,-122.42,United States of America,us
22425,"Kocaeli, İzmit, Türkiye",40.7663,29.9174,Türkiye,tr
37490,"Gueydan, LA, USA",30.0258,-92.509,United States of America,us
26728,"Paderborn, NW, Germany",51.7185,8.76538,Deutschland,de
5673,"Eskişehir, Turkey",39.6643,31.0757,Türkiye,tr
41055,"Rawalindi, Pakistan",30.3292,71.2475,پاکستان,pk


In [165]:
pd.set_option('display.max_rows', 500)

In [167]:
full_table_location['Location'].value_counts(sort = True).head(20)

India                          11789
Germany                         8300
London, United Kingdom          6835
United States                   6762
United Kingdom                  5191
Bangalore, Karnataka, India     5095
France                          4129
Paris, France                   3963
Netherlands                     3750
Bangalore, India                3575
Berlin, Germany                 3562
Canada                          3086
Pune, Maharashtra, India        2938
Israel                          2904
San Francisco, CA               2882
Moscow, Russia                  2600
Brazil                          2565
Singapore                       2426
Poland                          2417
Australia                       2343
Name: Location, dtype: int64

In [168]:
full_table_location['code'].value_counts(sort = True).head(20)

us    114383
in     69774
gb     33869
de     26968
ca     22979
fr     14163
au     12132
ru     10653
nl     10432
br     10287
pl      7938
es      7846
cn      7612
it      6796
se      6326
ua      5667
pk      5241
ch      4719
il      4468
be      4226
Name: code, dtype: int64

In [127]:
# Save as CSV just in case
merged_locations.to_csv('locationdata.csv', index=False)

In [135]:
# Left merge location data into active user data
full_table_location = pd.merge(activeusers, merged_locations, on='Location', how='left')

In [139]:
# Export
activeusers.to_csv('active_users.csv', index=False)